In [ ]:
# Start writing code here...

import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from igraph import *
import disjoint_set

This notebook contains template with example of data usage and loading (from prepared csv files, which were produced from original .xlsx file with the help of pandas)

In [ ]:
if not ".." in sys.path:
    sys.path.insert(0, "..")

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
os.listdir("../data")

['sources.csv.gz',
 'input_wagon.xlsx',
 'orders7.csv.gz',
 'orders8.csv.gz',
 'wagon_mode_compat.csv.gz',
 'description.txt',
 'output_wagon.xlsx',
 'metrics.csv.gz',
 'standplaces.csv.gz']

In [ ]:
orders7 = pd.read_csv("../data/orders7.csv.gz")
orders8 = pd.read_csv("../data/orders8.csv.gz")
wagon_mode_compat = pd.read_csv("../data/wagon_mode_compat.csv.gz")
sources = pd.read_csv("../data/sources.csv.gz")
standplaces = pd.read_csv("../data/standplaces.csv.gz")
metrics = pd.read_csv("../data/metrics.csv.gz")

In [ ]:
# load data from file (this gets checked and reloaded because of autoreload magic above)
from utils.common import *



SyntaxError: invalid syntax (common.py, line 161)

In [ ]:
# get_roads_graph


TypeError: '(slice(104200, 104210, None),)' is an invalid key

In [ ]:
from igraph import *
from utils.abstractions import get_roads_graph


graph, name2vertex = get_roads_graph(metrics)


0
50000
100000
150000
200000
250000
300000
350000
400000
450000
500000
550000
600000
650000
700000
750000
800000
850000
900000


In [ ]:
total_sum = 0

stale_price = {}
stale_move = {}
for index, row in sources.iterrows():
    stale_price[row['Station']] = int(row['Stand']) 


for index, row in metrics.iterrows():
    stale_move[str(row['From']) + "_" + str(row['To'])] = int(row['PriceUnit']) 
    stale_move[str(row['To']) + "_" + str(row['From'])] = int(row['PriceUnit']) 


In [ ]:
from utils.common import *

order_manager = OrderManager()
for index, row in orders7.iterrows():
    order = Order2(row['Start'], row['Finish'], row['StartDate'], int(row['MaxUnit']), row['NeedWagonModel'], int(row['Tariff'].split(":")[0]), row['Dur'], row['ShortagePenalty'], row['OrderNum'])
    order_manager.add_order(order)
    total_sum += order.total_profit()
print(total_sum)

wagon_manager = WagonManager()
for index, row in sources.iterrows():
    for units in range(row['Units']):
        occupied = True
        if int(row['Date']) == 0:
            occupied = False
        wagon = Wagon(row['WagonModel'], str(row['Station']), occupied, row['Date'])
        wagon_manager.add_wagon(wagon)

print(wagon_manager.wagon_number())


comp_wagon = set()
for index, row in wagon_mode_compat.iterrows():
    comp_wagon.add(str(row['NeedWagonModel']) + "_" + str(row['CompatibleWagonModel']))



strategy =  Strategy(order_manager, wagon_manager, graph, comp_wagon, name2vertex, stale_price, stale_move)
print(strategy.compability_model)
for day in range(28):
    strategy.run_day(day)
    print(day)
    #break
print("Bad orders - " +  str(strategy.bad_orders))
print("Total orders - " +  str(strategy.total_orders))

print("Profit - " + str(strategy.total_profit))
print("Stale stop - " + str(strategy.stale_stop))
print("Stale move - " + str(strategy.stale_move))
print("Profit per cat - " + str(strategy.profit_per_cat))


2174198750
15438
{'2309_2146', '2015_2133', '2456_2167', '2122_2061', '2491_2163', '2065_2049', '2345_2160', '2197_2158', '2362_2152', '2190_2164', '2221_2167', '2136_2161', '2025_2158', '2095_2091', '2424_2152', '2272_2164', '2272_2137', '2211_2144', '2393_2068', '2024_2071', '2025_2143', '2092_2157', '2069_2161', '2468_2149', '2458_2158', '2440_2140', '2304_2095', '2258_2066', '2013_2155', '2260_2187', '2265_2161', '2174_2143', '2112_2162', '2025_2037', '2120_2071', '2262_2133', '2284_2071', '2475_2162', '2391_2071', '2265_2044', '2176_2153', '2055_2191', '2253_2094', '2106_2155', '2298_2071', '2429_2184', '2169_2133', '2263_2095', '2438_2146', '2090_2162', '2002_2166', '2174_2153', '2218_2149', '2326_2166', '2031_2058', '2263_2040', '2281_2149', '2061_2068', '2176_2145', '2117_2071', '2286_2044', '2276_2149', '2001_2066', '2375_2062', '2437_2161', '2029_2071', '2031_2040', '2039_2053', '2452_2164', '2015_2167', '2277_2134', '2324_2052', '2127_2066', '2274_2154', '2253_2149', '2328_2

In [ ]:
strategy.stale_moves_num

8954

In [ ]:
with open("orders.csv", 'w') as orders:
    orders.write("Код заказа,Станция старта,Станция финиша,Расчетная дата старта,Плановая дата старта,Время в пути,Мин допустимый объем,Запрошенный объем,Тарифы,Запрошенный тип вагона,Приоритет,Примененный тариф\n")
    for order in strategy.ord_log:
        orders.write(order + "\n")
        


In [ ]:
with open("empty_moves.csv", 'w') as orders:
    orders.write("Отправление,Прибытие,Тип простоя,Модель вагона,День отправления (от начала расчета),День прибытия (от начала расчета),Количество,Цена порожнего пробега (за вагон),Код следующего заказa,Расчитанный тариф следующего заказа\n")
    for order in strategy.stale_move_log:
        orders.write(order + "\n")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7ba67d69-ce72-437b-8bf4-6d3c9b906c4a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>